# Scraping

### HTML request Setup

In [2]:
import requests
import bs4

spin_off_url = 'https://www.spin-off.fr/calendrier_des_series.html'
response = requests.get(spin_off_url).content
page = bs4.BeautifulSoup(response, 'html')

columns = page.find_all('td', class_='td_jour')

### Entity Episode Class

In [3]:
class Episode:

    def __init__(self, show, episode, season, country, channel, date, link):

        self.show = show
        self.episode = episode
        self.season = season
        self.country = country
        self.channel = channel
        self.date = date
        self.link = link
    

### Scraping Table columns

In [8]:
import re

calendar = [

    {
        'date': column.find('div', class_='div_jour').get('id')[5:],
        'names': [
            re.sub(' +saison [0-9]+ episode .+$', '', name.get('title')) for name in column.find_all('a', class_='liens')
        ],
        'episodes': [
            {
                'season': num[0],
                'episode': num[1]
            } for num in (name.getText().split('.') for name in column.find_all('a', class_='liens'))
        ],
        'countries': [
            country.find_previous_sibling().find_previous_sibling().get('alt') for country in column.find_all('span', class_='calendrier_episodes')
        ],
        'channels': [
            channel.find_previous_sibling().get('alt') for channel in column.find_all('span', class_='calendrier_episodes')
        ],
        'links': [
            link.get('href') for link in column.find_all('a', class_='liens')
        ]
    }
    for column in columns if column.find('div', class_='div_jour')

]


### Episodes formatting

In [9]:
episodes = []

for column in calendar:

    for i in range(0, len(column['names'])):

        episodes.append(

            Episode(

                column['names'][i],
                # Les épisodes spéciaux sont notés 'XX' et ne peuvent donc pas êtres convertis
                # en entiers, on utilisera donc -1
                int(column['episodes'][i]['episode']) if column['episodes'][i]['episode'] != 'XX' else -1,
                int(column['episodes'][i]['season']),
                column['countries'][i],
                column['channels'][i],
                column['date'],
                column['links'][i])
                
            )

print(len(episodes))


355


### CSV Export

In [10]:
import csv

with open('../datas/files/episodes.csv', 'w', encoding='utf-8', newline='') as csvfile:

    fieldnames = ['show', 'episode', 'season', 'country', 'channel', 'date', 'link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Écriture de l'en-tête du fichier CSV
    writer.writeheader()

    # Écriture des données des épisodes dans le fichier CSV
    for episode in episodes:
        writer.writerow({
            'show': episode.show,
            'episode': episode.episode,
            'season': episode.season,
            'country': episode.country,
            'channel': episode.channel,
            'date': episode.date,
            'link': episode.link,
        })

print('CSV file created successfuly')


CSV file created successfuly


### CSV Parsing

In [9]:

with open('../datas/files/episodes.csv', 'r', encoding='utf-8', newline='') as csvfile:

    # Ignorer l'en-tête du fichier CSV
    next(csvfile)

    table = []

    for line in csvfile:

        cells = line.split(',')

        show = cells[0]
        episode = cells[1]
        season = cells[2]
        country = cells[3]
        channel = cells[4]
        date = cells[5]
        link = cells[6]

        table.append((show, episode, season, country, channel, date, link))


print(table)


[('4 Estrellas', 110, 1, 'Espagne', 'TVE', '01-11-2023', 'episode110-411772-01112023-saison1-4-Estrellas.html\r\n'), ('Black Cake', 1, 1, 'Etats-Unis', 'Hulu', '01-11-2023', 'episode01-410147-01112023-saison1-Black-Cake.html\r\n'), ('Black Cake', 2, 1, 'Etats-Unis', 'Hulu', '01-11-2023', 'episode02-410148-01112023-saison1-Black-Cake.html\r\n'), ('Black Cake', 3, 1, 'Etats-Unis', 'Hulu', '01-11-2023', 'episode03-410149-01112023-saison1-Black-Cake.html\r\n'), ('Blanca', 5, 2, 'Italie', 'Rai 1', '01-11-2023', 'episode05-411693-01112023-saison2-Blanca.html\r\n'), ('Children Ruin Everything', 6, 3, 'Canada', 'CTV', '01-11-2023', 'episode06-411350-01112023-saison3-Children-Ruin-Everything.html\r\n'), ('Chucky', 5, 3, 'Etats-Unis', 'Syfy', '01-11-2023', 'episode05-409118-01112023-saison3-Chucky.html\r\n'), ("Cooper's Bar", 5, 2, 'Etats-Unis', 'AMC', '01-11-2023', 'episode05-411065-01112023-saison2-Cooper-s-Bar.html\r\n'), ('Demain nous appartient', 53, 7, 'France', 'TF1', '01-11-2023', 'episo

In [12]:

import sqlite3

conn = sqlite3.connect('../datas/database/database.db')
# Création d'un curseur pour exécuter des requêtes SQL
cursor = conn.cursor()

# Créer la table "episodes" dans la base de données
cursor.execute('''CREATE TABLE IF NOT EXISTS episodes
                  (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  show TEXT NOT NULL,
                  episode INTEGER NOT NULL,
                  season INTEGER NOT NULL,
                  country TEXT NOT NULL,
                  channel TEXT NOT NULL,
                  date TEXT NOT NULL,
                  link TEXT NOT NULL)''')

# Insérer l'objet Episode dans la table "episodes"
for episode in episodes:
    cursor.execute("INSERT INTO episodes (show, episode, season, country, channel, date, link) VALUES (?, ?, ?, ?, ?, ?, ?)",
                (episode.show, episode.episode, episode.season, episode.country, episode.channel,
                    episode.date, episode.link))

# Valider les changements et fermer la connexion
conn.commit()
conn.close()

print("Base de données créée avec succès.")

Base de données créée avec succès.


In [33]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('../datas/database/database.db')
cursor = conn.cursor()

# Requête SQL pour compter le nombre d'épisodes par chaîne en Octobre
query = """
    SELECT channel, COUNT(*) AS episode_count
    FROM episodes
    WHERE SUBSTR(date, 4, 2) = '11'
    GROUP BY channel
    ORDER BY episode_count DESC;
"""

# Exécution de la requête
cursor.execute(query)

# Récupération des résultats
results = cursor.fetchall()

# Affichage des résultats
for row in results:
    channel, episode_count = row
    print(f"Chaîne : {channel}, Nombre d'épisodes diffusés en Octobre : {episode_count}")

# Fermeture de la connexion à la base de données
conn.close()

Chaîne : Netflix, Nombre d'épisodes diffusés en Octobre : 39
Chaîne : Disney+, Nombre d'épisodes diffusés en Octobre : 32
Chaîne : Hulu, Nombre d'épisodes diffusés en Octobre : 24
Chaîne : BBC iPlayer, Nombre d'épisodes diffusés en Octobre : 20
Chaîne : Apple TV+, Nombre d'épisodes diffusés en Octobre : 17
Chaîne : NBC, Nombre d'épisodes diffusés en Octobre : 16
Chaîne : Paramount+, Nombre d'épisodes diffusés en Octobre : 15
Chaîne : Global, Nombre d'épisodes diffusés en Octobre : 13
Chaîne : HBO Max, Nombre d'épisodes diffusés en Octobre : 12
Chaîne : TF1, Nombre d'épisodes diffusés en Octobre : 10
Chaîne : Syfy, Nombre d'épisodes diffusés en Octobre : 9
Chaîne : Fox, Nombre d'épisodes diffusés en Octobre : 9
Chaîne : ZDF, Nombre d'épisodes diffusés en Octobre : 7
Chaîne : CraveTV, Nombre d'épisodes diffusés en Octobre : 7
Chaîne : CBC, Nombre d'épisodes diffusés en Octobre : 6
Chaîne : AMC, Nombre d'épisodes diffusés en Octobre : 6
Chaîne : Prime Video, Nombre d'épisodes diffusés en 

In [36]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('../datas/database/database.db')
cursor = conn.cursor()

# Requête SQL pour compter le nombre d'épisodes par chaîne en Octobre
query = """
    SELECT country, COUNT(*) AS episode_count
    FROM episodes
    WHERE SUBSTR(date, 4, 2) = '11'
    GROUP BY country
    ORDER BY episode_count DESC;
"""

# Exécution de la requête
cursor.execute(query)

# Récupération des résultats
resultsBis = cursor.fetchall()

# Affichage des résultats
for row in resultsBis:
    country, episode_count = row
    print(f"{country}, Nombre d'épisodes diffusés en Octobre : {episode_count}")

# Fermeture de la connexion à la base de données
conn.close()

Etats-Unis, Nombre d'épisodes diffusés en Octobre : 238
Canada, Nombre d'épisodes diffusés en Octobre : 43
Royaume-Uni, Nombre d'épisodes diffusés en Octobre : 31
France, Nombre d'épisodes diffusés en Octobre : 20
Espagne, Nombre d'épisodes diffusés en Octobre : 7
Allemagne, Nombre d'épisodes diffusés en Octobre : 7
Suède, Nombre d'épisodes diffusés en Octobre : 4
Italie, Nombre d'épisodes diffusés en Octobre : 3
Turquie, Nombre d'épisodes diffusés en Octobre : 1
Belgique, Nombre d'épisodes diffusés en Octobre : 1


In [40]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('../datas/database/database.db')
cursor = conn.cursor()

# Requête SQL pour compter le nombre d'épisodes par chaîne en Octobre
query = """
    SELECT show 
    FROM episodes
    WHERE SUBSTR(date, 4, 2) = '11'
    GROUP BY show;
"""

# Exécution de la requête
cursor.execute(query)

# Récupération des résultats
resultsTer = cursor.fetchall()
conn.close()

# Récupération des mots
dictionary = {}

for row in resultsTer:
    show, = row

    for word in show.split(' '):
        
        isNotWord = re.search('[0-9()[\\]_]', word)
        if isNotWord:
            continue
        
        word = word.lower()
        
        if not word in dictionary:
            dictionary[word] = 0
        dictionary[word] += 1
    
sorted_counter = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True))

print(sorted_counter)

{'the': 21, 'all': 4, 'of': 3, 'and': 3, 'de': 3, 'murder': 2, 'at': 2, 'good': 2, 'legacy': 2, 'family': 2, 'tout': 2, 'la': 2, 'soko': 2, 'un': 2, 'estrellas': 1, 'a': 1, 'end': 1, 'world': 1, 'acting': 1, 'creatures': 1, 'great': 1, 'small': 1, 'rise': 1, 'light': 1, 'we': 1, 'cannot': 1, 'see': 1, 'american': 1, 'dad!': 1, 'anderson': 1, 'spider': 1, 'silva': 1, 'andy': 1, 'band': 1, 'moment': 1, 'shi': 1, 'ci': 1, 'beacon': 1, 'billy': 1, 'kid': 1, 'black': 1, 'cake': 1, 'blackberry': 1, 'blanca': 1, "bob's": 1, 'burgers': 1, 'boomer': 1, 'bosch:': 1, 'bros': 1, 'children': 1, 'ruin': 1, 'everything': 1, 'chucky': 1, "cooper's": 1, 'bar': 1, 'culprits': 1, "d'argent": 1, 'et': 1, 'sang': 1, 'dna': 1, 'do': 1, 'crime': 1, 'demain': 1, 'nous': 1, 'appartient': 1, 'deutsches': 1, 'haus': 1, 'doctor': 1, 'who': 1, 'doom': 1, 'patrol': 1, 'en': 1, 'helt': 1, 'vanlig': 1, 'familj': 1, 'entre': 1, 'tierras': 1, 'guy': 1, 'law': 1, 'fargo': 1, 'fear': 1, 'walking': 1, 'dead': 1, 'fellow':